In [ ]:
# by default, the tf.layers.dense() function uses Xavier intialization (with a unform distribution)
# it could be changed to He intialization by using the variance_scaling_intializer()


he_init = tf.contrib.layers.variance_scaling_intializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.relu, 
                         kernel_intializer = he_init, name = 'hidden1')

In [ ]:
# using the elu as activation function for a simple neural net
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.elu, name = 'hidden1')


def leaky_relu(z, name = None):
    return tf.maximum(0.01 *z, z, name = name)
# using the leaky_relu as activation function for a simple neural net
hidden1 = tf.layers.dense(X, n_hidden1, activation = leaky_relu, name = 'hidden1')

In [ ]:
# Batch Normalization
